In [1]:
import sys
sys.path.append('./..')
sys.path.append('./img2img')

import argparse
import datetime
import os
from time import time
from collections import OrderedDict

import numpy as np
import torch
from torchvision.models.resnet import resnet50
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

from src.datasets import get_dataset, DATASETS, get_num_classes, get_corrupted_dataset, load_parallel_model
from src.img2img.img2img_models import img2img_get_model, get_size
from src.core import Smooth, TSmooth
from src.img2img.corruption import Corruption
from src.architectures import get_architecture
from some_our_code.utils import make_our_dataset_v2, CustomAudioDataset
from archs.cifar_resnet import resnet as resnet_cifar

sys.path.append('./../../')
# from architectures import get_architecture
from datasets_utils import get_normalize_layer


# import setGPU
# from src.grad2n import gradnorm_batch, gradnorm_batch_reduced
# from src.analyze import plot_certified_accuracy, Line, ApproximateAccuracy

In [2]:
!nvidia-smi

Thu Feb  1 17:27:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           On  | 00000000:06:00.0 Off |                    0 |
| N/A   39C    P0              60W / 300W |  12447MiB / 16384MiB |      1%   E. Process |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
%env CUDA_VISIBLE_DEVICES=2
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_VISIBLE_DEVICES=2
env: CUDA_LAUNCH_BLOCKING=1


In [4]:
torch.tensor([1]).to("cuda")

tensor([1], device='cuda:0')

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
parser = argparse.ArgumentParser(description='Certify many examples')
parser.add_argument("--dataset",
                    default='cifar10',choices=DATASETS, help="which dataset")
parser.add_argument("--base_classifier", type=str,

                    default=None,
                    help="path to saved pytorch model of base classifier")
# parser.add_argument("sigma", type=float, help="noise hyperparameter")
parser.add_argument("--outfile", type=str,
                    default='./logs/certify_results',
                    help="output file")


parser.add_argument("--batch", type=int, default=512, help="batch size")
parser.add_argument("--skip", type=int, default=30
                    , help="how many examples to skip")
parser.add_argument("--max", type=int, default=-1, help="stop after this many examples")
parser.add_argument("--split", choices=["train", "test"], default="test", help="train or test set")
parser.add_argument("--N0", type=int, default=200)
parser.add_argument("--N", type=int, default=1000, help="number of samples to use")
parser.add_argument("--alpha", type=float, default=1e-3, help="failure probability")





parser.add_argument('--corrupt',type=str,default=['none','gaussian_blur','motion_blur','zoom_blur','rotate','translate','contrast','pixelate','jpeg',][1],
                    help=' The corruption type for training')
parser.add_argument('--add_noise',type=float, default=0.0)
parser.add_argument('--noise_dst',default=["none","gaussian","exp","uniform","folded_gaussian"][2],type=str)
parser.add_argument('--noise_sd', default=0.8, type=float,
                    help="standard deviation of Gaussian noise for data augmentation")


parser.add_argument('--partial_min',default=0.0, type=float,
                    help = "Minimal of certify range")
parser.add_argument('--partial_max',default=1.0, type=float,
                    help = "Maximal of certify range")



parser.add_argument('--arch', type=str,
                    default=["edsr","unet","runet"][2])

args = parser.parse_args([])

In [7]:
args

Namespace(N=1000, N0=200, add_noise=0.0, alpha=0.001, arch='runet', base_classifier=None, batch=512, corrupt='gaussian_blur', dataset='cifar10', max=-1, noise_dst='exp', noise_sd=0.8, outfile='./logs/certify_results', partial_max=1.0, partial_min=0.0, skip=30, split='test')

In [8]:
NUM_IMAGES_FOR_TEST = 500

def make_test_dataset(args):
    test_dataset = get_dataset(args.dataset, "test")
    pin_memory = (args.dataset == "imagenet")
    np.random.seed(42)
    idxes = np.random.choice(len(test_dataset), NUM_IMAGES_FOR_TEST, replace=False)
    
    ourdataset = make_our_dataset_v2(test_dataset, idxes)
    ourdataloader = DataLoader(ourdataset, shuffle=False, batch_size=1,
                         num_workers=6, pin_memory=False)
    return ourdataset, ourdataloader

In [9]:
test_dataset = get_dataset('imagenet', 'test')
pin_memory = False  # ('imagenet' == "imagenet")
bs = 64
np.random.seed(42)
NUM_IMAGES_FOR_TEST = 500
idxes = np.random.choice(len(test_dataset), NUM_IMAGES_FOR_TEST , replace=False)

print(idxes[:10])

[33553  9427   199 12447 39489 42724 10822 49498  4144 36958]


In [10]:
ourdataset500 = make_our_dataset_v2(test_dataset, idxes)

dataloader500 = DataLoader(ourdataset500, shuffle=False, batch_size=1,
                         num_workers=6, pin_memory=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:11<00:00, 43.02it/s]


In [11]:
np.random.seed(42)
NUM_IMAGES_FOR_TEST = 500
cifar10_ds = get_dataset('cifar10', 'test')
idxes2 = np.random.choice(len(cifar10_ds), NUM_IMAGES_FOR_TEST , replace=False)

our_cifar10_500 = make_our_dataset_v2(cifar10_ds, idxes2)
dataloader_cifar10 = DataLoader(our_cifar10_500, shuffle=False, batch_size=1, num_workers=6, pin_memory=False)

Files already downloaded and verified


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 5589.35it/s]


In [12]:
np.random.seed(42)

cifar100_ds = get_dataset('cifar100', 'test')
idxes3 = np.random.choice(len(cifar100_ds), NUM_IMAGES_FOR_TEST , replace=False)

our_cifar100_500 = make_our_dataset_v2(cifar100_ds, idxes3)
dataloader_cifar100 = DataLoader(our_cifar100_500, shuffle=False, batch_size=1,
                         num_workers=6, pin_memory=False)

Files already downloaded and verified


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 6081.27it/s]


# Contrast
# or is it Contrast-Brightness(??) according to the code

In [13]:
args.corrupt = 'contrast'
args.noise_sd = 0.3
args.noise_dst = "gaussian"
args.partial_max = 0.4

## CIFAR-10

In [14]:
args.base_classifier = "../../tss_weights/cifar10/resnet110/brightness-contrast/0.3_0.3/consistency/checkpoint.pth.tar"
# args.base_classifier = "../../new_results/cifar100_resnet110_cifar100_cb_cifar100_cb_checkpoint.pth.tar"

args.dataset = 'cifar10'

dataset = our_cifar10_500



device = torch.device('cuda')

checkpoint = torch.load(args.base_classifier, map_location=device)
base_classifier = get_architecture(checkpoint["arch"], args.dataset, device=device)
base_classifier.load_state_dict(checkpoint["state_dict"])


img_size, param_size = get_size(args)

model_t = img2img_get_model(args, img_size, param_size,parallel=False)

trans_path = None
if trans_path is not None:
    state_dict = torch.load(trans_path)['state_dict']
    model_t.load_state_dict(state_dict)
# model_t = load_parallel_model(model_t, state_dict)

# to device
base_classifier.to(device)
base_classifier.eval()
model_t.to(device)
model_t.eval()


#load dataset

# corruptor
corruptor = Corruption(args, co_type=args.corrupt,add_noise=args.add_noise,noise_sd=args.noise_sd,distribution=args.noise_dst)

# create the smooothed classifier g
smoothed_classifier = TSmooth(base_classifier, None, corruptor, get_num_classes(args.dataset),args.noise_dst,args.noise_sd, args.add_noise)

# prepare output file
f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime", file=f, flush=True)
print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime")

tot, tot_good, tot_correct = 0, 0, 0

# for gaussian smooth
attack_radius = args.partial_max


# dataset = get_dataset(args.dataset, args.split)
dataset = our_cifar10_500
for i in tqdm(range(len(dataset))):

#     # only certify every args.skip examples, and stop after args.max examples
#     if i % args.skip != 0:
#         continue
#     if i == args.max:
#         break

    (x, label) = dataset[i]

    before_time = time()
    x = x.to(device)
    prediction, radius = smoothed_classifier.certify(x, args.N0, args.N, args.alpha, args.batch)


    correct = (prediction == label).item()
    cond1 = radius * args.noise_sd > args.partial_max
#     print(cond1.dtype)
#     print(correct.dtype)
    good = (radius * args.noise_sd > args.partial_max)&correct

    tot, tot_good, tot_correct = tot+1, tot_good+good, tot_correct+correct
    after_time = time()
    time_elapsed = str(datetime.timedelta(seconds=(after_time - before_time)))
    print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed))

f.close()

print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot))

f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot), file=f, flush=True)
f.close()

idx	label	predict	radius	good	correct	time


  0%|▎                                                                                                                                        | 1/500 [00:05<49:08,  5.91s/it]

0	2	2	1.56121	True	True	0:00:05.909347


  0%|▌                                                                                                                                        | 2/500 [00:11<48:20,  5.83s/it]

1	1	1	2.46326	True	True	0:00:05.763143


...

497	1	1	2.46326	True	True	0:00:06.153129


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 499/500 [50:10<00:06,  6.05s/it]

498	0	0	2.46326	True	True	0:00:06.002756


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [50:16<00:00,  6.03s/it]

499	8	8	2.46326	True	True	0:00:06.043417
Total 500 Certified 428 Certified Acc 0.856 Test Acc 0.868


In [15]:
rrrr = (tot, tot_good, tot_good/tot, tot_correct/tot)


In [16]:
print(rrrr[2])

0.856


## CIFAR-100

In [31]:
# args.base_classifier = "../../tss_weights/cifar10/resnet110/brightness-contrast/0.3_0.3/consistency/checkpoint.pth.tar"
args.base_classifier = "../../new_results/cifar100_resnet110_cifar100_cb_cifar100_cb_checkpoint.pth.tar"
args.corrupt = 'contrast'
args.dataset = 'cifar100'
args.noise_sd = 0.3
dataset = our_cifar100_500



# device = torch.device('cuda')
# checkpoint = torch.load(args.base_classifier, map_location=device)
# base_classifier = get_architecture(checkpoint["arch"], args.dataset)
# base_classifier.load_state_dict(checkpoint["state_dict"])


model = get_architecture(arch="cifar100_resnet110", dataset="cifar100", device=device)
checkpoint = torch.load(args.base_classifier, map_location=device)
model.load_state_dict(checkpoint['state_dict'])
model.eval()
model = model.to(device)

base_classifier = model





# load the base classifier

# args.base_classifier = "../../tss_weights/cifar10/resnet110/brightness-contrast/0.4_0.4/consistency/checkpoint.pth.tar"





img_size, param_size = get_size(args)

model_t = img2img_get_model(args, img_size, param_size,parallel=False)

trans_path = None
if trans_path is not None:
    state_dict = torch.load(trans_path)['state_dict']
    model_t.load_state_dict(state_dict)
# model_t = load_parallel_model(model_t, state_dict)

# to device
base_classifier.to(device)
base_classifier.eval()
model_t.to(device)
model_t.eval()


#load dataset

# corruptor
corruptor = Corruption(args, co_type=args.corrupt,add_noise=args.add_noise,noise_sd=args.noise_sd,distribution=args.noise_dst)

# create the smooothed classifier g
smoothed_classifier = TSmooth(base_classifier, None, corruptor, get_num_classes(args.dataset),args.noise_dst,args.noise_sd, args.add_noise)

# prepare output file
f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime", file=f, flush=True)
print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime")

tot, tot_good, tot_correct = 0, 0, 0

# for gaussian smooth
attack_radius = args.partial_max




for i in tqdm(range(len(dataset))):

#     # only certify every args.skip examples, and stop after args.max examples
#     if i % args.skip != 0:
#         continue
#     if i == args.max:
#         break

    (x, label) = dataset[i]

    before_time = time()
    x = x.to(device)
    prediction, radius = smoothed_classifier.certify(x, args.N0, args.N, args.alpha, args.batch)


    correct = (prediction == label).item()
    cond1 = radius * args.noise_sd > args.partial_max
#     print(cond1.dtype)
#     print(correct.dtype)
    good = (radius * args.noise_sd > args.partial_max)&correct

    tot, tot_good, tot_correct = tot+1, tot_good+good, tot_correct+correct
    after_time = time()
    time_elapsed = str(datetime.timedelta(seconds=(after_time - before_time)))
    print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed))

f.close()

print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot))

f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot), file=f, flush=True)
f.close()

idx	label	predict	radius	good	correct	time


  0%|▏                                                                                         | 1/500 [00:08<1:09:05,  8.31s/it]

0	10	10	0.84282	False	True	0:00:08.307408


  0%|▎                                                                                         | 2/500 [00:16<1:09:23,  8.36s/it]

1	46	62	0.05969	False	False	0:00:08.394137


...

497	67	67	2.46326	True	True	0:00:08.503971


100%|███████████████████████████████████████████████████████████████████████████████████████▊| 499/500 [1:11:40<00:08,  8.88s/it]

498	74	78	2.46326	False	False	0:00:08.803202


100%|████████████████████████████████████████████████████████████████████████████████████████| 500/500 [1:11:48<00:00,  8.62s/it]

499	29	95	0.54053	False	False	0:00:08.314465
Total 500 Certified 207 Certified Acc 0.414 Test Acc 0.47


In [ ]:
2

In [23]:
## Gsmooth

print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot))

Total 500 Certified 216 Certified Acc 0.432 Test Acc 0.47


In [15]:


f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot), file=f, flush=True)
f.close()

## Imagenet

In [6]:
args.corrupt = "contrast"
args.noise_sd = 0.6
args.noise_dst = "gaussian"
args.partial_max = 0.4
args.dataset = "imagenet"
args.base_classifier = "../../checkpoints/brightness/checkpoint.pth.tar"
args.device = "cpu"
args.arch = "resnet50"



In [9]:
def calculate_gsmooth(args):
    
    device = torch.device(args.device)
    model = get_architecture(arch=args.arch, dataset=args.dataset, device=device)
    checkpoint = torch.load(args.base_classifier, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    model = model.to(device)

    base_classifier = model
    
    dataset, _ = make_test_dataset(args)

#     img_size, param_size = get_size(args)

#     model_t = img2img_get_model(args, img_size, param_size,parallel=False)

#     trans_path = None
#     if trans_path is not None:
#         state_dict = torch.load(trans_path)['state_dict']
#         model_t.load_state_dict(state_dict)
#     # model_t = load_parallel_model(model_t, state_dict)
    
#     model_t.to(device)
#     model_t.eval()

    # to device
    base_classifier.to(device)
    base_classifier.eval()


    #load dataset

    # corruptor
    corruptor = Corruption(args, co_type=args.corrupt,add_noise=args.add_noise,noise_sd=args.noise_sd,distribution=args.noise_dst)

    # create the smooothed classifier g
    smoothed_classifier = TSmooth(base_classifier, None, corruptor, get_num_classes(args.dataset),args.noise_dst,args.noise_sd, args.add_noise)

    # prepare output file
    f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd)+'_running', 'w')
    print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime", file=f, flush=True)
    print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime")

    tot, tot_good, tot_correct = 0, 0, 0

    # for gaussian smooth
    attack_radius = args.partial_max
    for i in tqdm(range(len(dataset))):
        (x, label) = dataset[i]

        before_time = time()
        x = x.to(device)
        prediction, radius = smoothed_classifier.certify(x, args.N0, args.N, args.alpha, args.batch)


        correct = (prediction == label).item()
        cond1 = radius * args.noise_sd > args.partial_max
        good = (radius * args.noise_sd > args.partial_max)&correct

        tot, tot_good, tot_correct = tot+1, tot_good+good, tot_correct+correct
        after_time = time()
        time_elapsed = str(datetime.timedelta(seconds=(after_time - before_time)))
        print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed), file=f, flush=True)
        print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed))

    f.close()

    print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot))

    f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd)+'_total_result', 'w')
    print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot), file=f, flush=True)
    f.close()


In [10]:
calculate_gsmooth(args)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 97.79it/s]


idx	label	predict	radius	good	correct	time


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:58<00:00, 118.78s/it]

0	671	671	2.46326	True	True	0:01:58.777288
Total 1 Certified 1 Certified Acc 1.0 Test Acc 1.0


In [13]:
import yaml 

from yaml import load

In [15]:
%%writefile sample_config.yaml
corrupt: "contrast"
noise_sd: 0.6
noise_dst: "gaussian"
partial_max: 0.4
dataset: "imagenet"
base_classifier: "../../checkpoints/brightness/checkpoint.pth.tar"
device: "cpu"
arch: "resnet50"

Overwriting sample_config.yaml


In [19]:
config = load("sample_config.yaml", Loader=yaml.FullLoader)
config

'sample_config.yaml'

In [25]:
with open("sample_config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [26]:
config["corrupt"]

'contrast'

In [27]:
config

{'corrupt': 'contrast',
 'noise_sd': 0.6,
 'noise_dst': 'gaussian',
 'partial_max': 0.4,
 'dataset': 'imagenet',
 'base_classifier': '../../checkpoints/brightness/checkpoint.pth.tar',
 'device': 'cpu',
 'arch': 'resnet50'}

In [1]:
%%writefile gsmooth_certify_resolvable.py
import sys
sys.path.append('./..')
sys.path.append('./img2img')

import os
os.environ['CUDA_VISIBLE_DEVICES']="5"
os.environ['CUDA_LAUNCH_BLOCKING']="1"

import argparse
import datetime

from time import time
from collections import OrderedDict
import yaml

import click
import numpy as np
import torch
from torchvision.models.resnet import resnet50
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

from src.datasets import get_dataset, DATASETS, get_num_classes, get_corrupted_dataset, load_parallel_model
from src.img2img.img2img_models import img2img_get_model, get_size
from src.core import Smooth, TSmooth
from src.img2img.corruption import Corruption
from src.architectures import get_architecture
from some_our_code.utils import make_our_dataset_v2, CustomAudioDataset
from archs.cifar_resnet import resnet as resnet_cifar

sys.path.append('./../../')
# from architectures import get_architecture
from datasets_utils import get_normalize_layer


NUM_IMAGES_FOR_TEST = 500


parser = argparse.ArgumentParser(description='Certify many examples')
parser.add_argument("--dataset",
                    default='cifar10',choices=DATASETS, help="which dataset")
parser.add_argument("--base_classifier", type=str,
                    default=None,
                    help="path to saved pytorch model of base classifier")
# parser.add_argument("sigma", type=float, help="noise hyperparameter")
parser.add_argument("--outfile", type=str,
                    default='./logs/certify_results',
                    help="output file")
parser.add_argument("--batch", type=int, default=512, help="batch size")
parser.add_argument("--skip", type=int, default=30
                    , help="how many examples to skip")
parser.add_argument("--max", type=int, default=-1, help="stop after this many examples")
parser.add_argument("--split", choices=["train", "test"], default="test", help="train or test set")
parser.add_argument("--N0", type=int, default=100)
parser.add_argument("--N", type=int, default=1000, help="number of samples to use")
parser.add_argument("--alpha", type=float, default=1e-3, help="failure probability")
parser.add_argument('--corrupt',type=str,default=['none','gaussian_blur','motion_blur','zoom_blur','rotate','translate','contrast','pixelate','jpeg',][1],
                    help=' The corruption type for training')
parser.add_argument('--add_noise',type=float, default=0.0)
parser.add_argument('--noise_dst',default=["none","gaussian","exp","uniform","folded_gaussian"][2],type=str)
parser.add_argument('--noise_sd', default=0.8, type=float,
                    help="standard deviation of Gaussian noise for data augmentation")
parser.add_argument('--partial_min',default=0.0, type=float,
                    help = "Minimal of certify range")
parser.add_argument('--partial_max',default=1.0, type=float,
                    help = "Maximal of certify range")
parser.add_argument('--arch', type=str, default=["edsr","unet","runet"][2])
args = parser.parse_args([])


def make_test_dataset(args):
    test_dataset = get_dataset(args.dataset, "test")
    pin_memory = (args.dataset == "imagenet")
    np.random.seed(42)
    idxes = np.random.choice(len(test_dataset), NUM_IMAGES_FOR_TEST, replace=False)
    
    ourdataset = make_our_dataset_v2(test_dataset, idxes)
    ourdataloader = DataLoader(ourdataset, shuffle=False, batch_size=1,
                         num_workers=6, pin_memory=False)
    return ourdataset, ourdataloader



def calculate_gsmooth(args):
    
    device = torch.device(args.device)
    model = get_architecture(arch=args.arch, dataset=args.dataset, device=device)
    checkpoint = torch.load(args.base_classifier, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    model = model.to(device)

    base_classifier = model
    
    dataset, _ = make_test_dataset(args)
    
    base_classifier.to(device)
    base_classifier.eval()

    corruptor = Corruption(args, co_type=args.corrupt,add_noise=args.add_noise,noise_sd=args.noise_sd,distribution=args.noise_dst)

    # create the smooothed classifier g
    smoothed_classifier = TSmooth(base_classifier, None, corruptor, get_num_classes(args.dataset),args.noise_dst,args.noise_sd, args.add_noise)

    # prepare output file
    filename = args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd) +"_" +str(args.partial_max)
    f = open(filename+'_running', 'w')
    print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime", file=f, flush=True)
    print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime")

    tot, tot_good, tot_correct = 0, 0, 0

    # for gaussian smooth
    attack_radius = args.partial_max
    for i in tqdm(range(len(dataset))):
        (x, label) = dataset[i]

        before_time = time()
        x = x.to(device)
        prediction, radius = smoothed_classifier.certify(x, args.N0, args.N, args.alpha, args.batch)


        correct = (prediction == label).item()
        cond1 = radius * args.noise_sd > args.partial_max
        good = (radius * args.noise_sd > args.partial_max)&correct

        tot, tot_good, tot_correct = tot+1, tot_good+good, tot_correct+correct
        after_time = time()
        time_elapsed = str(datetime.timedelta(seconds=(after_time - before_time)))
        print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed), file=f, flush=True)
        print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed))

    f.close()

    print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot))

    f = open(filename+'_total_result', 'w')
    print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot), file=f, flush=True)
    f.close()

    
@click.command()
@click.argument("config_path", type=click.Path(exists=True))
def main(config_path):
    with open(config_path, "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
        
    args.corrupt = config["corrupt"]
    args.noise_sd = config["noise_sd"]
    args.noise_dst = config["noise_dst"]
    args.partial_max = config["partial_max"]
    args.dataset = config["dataset"]
    args.base_classifier = config["base_classifier"]
    args.device = config["device"]
    args.arch = config["arch"]
    
#     if config["gpu"]:
#         os.environ['CUDA_VISIBLE_DEVICES']=str(config["gpu"])
#         os.environ['CUDA_LAUNCH_BLOCKING']="1"
        


    calculate_gsmooth(args)

if __name__ == "__main__":
    main()

Overwriting gsmooth_certify_resolvable.py


In [ ]:
!python gsmooth_certify_resolvable.py sample_config.yaml

100%|█████████████████████████████████████████| 500/500 [00:12<00:00, 41.62it/s]
idx	label	predict	radius	good	correct	time
 25%|█████████▌                             | 123/500 [43:40<1:32:32, 14.73s/it]

In [ ]:
!python gsmooth_certify_resolvable.py configs/imagenet_trans.yaml

In [ ]:
!python gsmooth_certify_resolvable.py configs/imagenet_brightness.yaml

In [ ]:
!python gsmooth_certify_resolvable.py configs/imagenet_contrast.yaml

In [ ]:
!python gsmooth_certify_resolvable.py configs/imagenet_blur.yaml